In [ ]:
# Use %pip so it binds to the current kernel
%pip -q uninstall -y transformers tokenizers diffusers huggingface-hub sentence-transformers peft gradio torchtune > /dev/null 2>&1


In [ ]:
# Install the missing dependency and align the core HF/Diffusers stack
%pip -q install --upgrade pip
%pip -q install "huggingface_hub==0.33.5"

# Make sure the rest of the stack is compatible with hub 0.33.5
%pip -q install --upgrade \
  "transformers==4.46.2" \
  "accelerate==0.31.0" \
  "diffusers==0.31.0" \
  "safetensors>=0.4.3" \
  "pillow>=10.3.0" \
  "xformers>=0.0.27"

import transformers, diffusers, huggingface_hub
print("transformers:", transformers.__version__)
print("diffusers:", diffusers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)


transformers: 4.46.2
diffusers: 0.31.0
huggingface_hub: 0.33.5


# Imports, folders, helpers

In [ ]:
import os, random
from pathlib import Path
import torch
from PIL import Image

OUTPUT_DIR = Path("outputs_sd"); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def set_seed(seed=42):
    random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed(42)

def save_image(img: Image.Image, name: str) -> str:
    path = str(OUTPUT_DIR / name)
    img.save(path)
    print("saved ->", path)
    return path

print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


CUDA available: True
GPU: Tesla T4


# Load Stable Diffusion (v1.5; SDXL optional)

In [ ]:
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline

MODEL_ID = "runwayml/stable-diffusion-v1-5"   # swap if you have a strong GPU:
# MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)  # keep simple to avoid offload/low_cpu_mem kwargs

if device == "cuda":
    pipe.enable_attention_slicing()
    pipe.to(device)

print("pipeline ready on:", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

pipeline ready on: cuda


# Project-adapted prompts (Graph-RAG + Agents + Trust/BDI + ReliScore UI)

In [ ]:
negative_prompt = "low quality, blurry, nsfw, text watermark, logo, distorted text"

prompts = [
    # Combined poster: Graph-RAG + agents + trust + ReliScore UI
    ("a product architecture card for a medical text app with tabs Summarize and Write, "
     "Graph-RAG evidence cards with citations, PHI sanitizer badge, trust score gauge with BDI icons, "
     "agent planner/executor swimlane, Stable Diffusion tool node, IMRaD template blocks, "
     "rounded cards, subtle shadows, teal+indigo accents, clean typography, flat vector illustration"),

    # MDAgents-style triage (recruiter → MDT → ICT across complexity bands)
    ("clean isometric diagram of a multi-agent medical decision pipeline, "
     "triage by complexity (low/moderate/high), recruiter → MDT → ICT arrows, "
     "subtle medical icons, teal+indigo UI, flat vector illustration, presentation-ready"),

    # Trust/BDI infographic
    ("an infographic of a trust game with trustor→trustee flow, BDI panels "
     "(belief, desire, intention), reciprocity and risk gauges, modern flat infographic, "
     "clean typography, labeled sections, print-ready")
]

generated_paths = []
for i, p in enumerate(prompts):
    img = pipe(
        prompt=p,
        negative_prompt=negative_prompt,
        num_inference_steps=25,
        guidance_scale=7.5,
        height=512,
        width=512
    ).images[0]
    generated_paths.append(save_image(img, f"txt2img_{i}.png"))

generated_paths


  0%|          | 0/25 [00:00<?, ?it/s]

saved -> outputs_sd/txt2img_0.png


  0%|          | 0/25 [00:00<?, ?it/s]

saved -> outputs_sd/txt2img_1.png


  0%|          | 0/25 [00:00<?, ?it/s]

saved -> outputs_sd/txt2img_2.png


['outputs_sd/txt2img_0.png',
 'outputs_sd/txt2img_1.png',
 'outputs_sd/txt2img_2.png']

# E) (Optional) One edit (img2img) — night-neon variant of image #0

In [ ]:
init_image = Image.open(generated_paths[0]).convert("RGB").resize((512, 512))

img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    MODEL_ID, torch_dtype=torch.float16 # Changed from pipe.torch_dtype to torch.float16
).to(device)

edit_prompt = ("night mode re-style with neon cyan/violet accents, gentle bloom, "
               "subtle hospital UI overlays, crisp vector edges, high readability")

edited = img2img(
    prompt=edit_prompt,
    image=init_image,
    strength=0.6,
    guidance_scale=8.0,
    num_inference_steps=30
).images[0]

save_image(edited, "img2img_edit.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

saved -> outputs_sd/img2img_edit.png


'outputs_sd/img2img_edit.png'

# Prepare a tiny LoRA dataset (10–20 SAFE images) + start training

In [ ]:
from pathlib import Path
from PIL import Image, ImageEnhance, ImageOps
import random, shutil, os

LORA_DATA = Path("lora_data")
LORA_OUT  = Path("lora_weights")
LORA_DATA.mkdir(parents=True, exist_ok=True)
LORA_OUT.mkdir(parents=True, exist_ok=True)

# If user hasn't uploaded anything, auto-populate with augmented copies (demo only)
existing = list(LORA_DATA.glob("*.png")) + list(LORA_DATA.glob("*.jpg")) + list(LORA_DATA.glob("*.jpeg"))
if len(existing) < 10:
    print("ℹ️ lora_data/ has fewer than 10 images — creating an augmented demo set from your generated images.")
    srcs = [Path(p) for p in generated_paths]  # from earlier txt2img
    if len(srcs) == 0:
        raise RuntimeError("No generated images found. Re-run the txt2img block first.")
    target_n = 12
    i = 0
    for src in srcs:
        img = Image.open(src).convert("RGB")
        for _ in range(4):  # 4 variants per src
            # Simple, safe augmentations
            v = img.copy().resize((512,512))
            if random.random() < 0.5:
                v = ImageOps.mirror(v)
            if random.random() < 0.5:
                v = ImageEnhance.Brightness(v).enhance(0.9 + 0.2*random.random())
            if random.random() < 0.5:
                v = ImageEnhance.Contrast(v).enhance(0.9 + 0.2*random.random())
            out = LORA_DATA / f"demo_{i:02d}.png"
            v.save(out)
            i += 1
            if i >= target_n:
                break
        if i >= target_n:
            break

print("✅ LoRA data dir:", LORA_DATA.resolve())
print("   Images:", len(list(LORA_DATA.glob('*'))))
print("✅ LoRA out dir:", LORA_OUT.resolve())


ℹ️ lora_data/ has fewer than 10 images — creating an augmented demo set from your generated images.
✅ LoRA data dir: /content/lora_data
   Images: 12
✅ LoRA out dir: /content/lora_weights


### Train a tiny LoRA (proof run)

In [ ]:
# Tiny proof run (≈10–15 min on T4). Increase max_train_steps to 800–1200 for better style locking.
!accelerate launch -m diffusers.examples.text_to_image.train_text_to_image_lora \
  --pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5 \
  --instance_data_dir=lora_data \
  --output_dir=lora_weights \
  --train_batch_size=1 --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 --lr_warmup_steps=0 \
  --max_train_steps=400 --mixed_precision="fp16"


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/bin/python3: Error while finding module specification for 'diffusers.examples.text_to_image.train_text_to_image_lora' (ModuleNotFoundError: No module named 'diffusers.examples')
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 7, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/launch.py", line 1097, in launch_command
    simple_launcher(args)
  File "/usr/local/lib/python3.12/dist-pac

In [ ]:
# Fallback: call by file path (installed with diffusers)
import inspect, diffusers, os, sys
script_path = os.path.join(os.path.dirname(inspect.getfile(diffusers)), "examples/text_to_image/train_text_to_image_lora.py")
print("Using script:", script_path)
!accelerate launch {script_path} \
  --pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5 \
  --instance_data_dir=lora_data \
  --output_dir=lora_weights \
  --train_batch_size=1 --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 --lr_warmup_steps=0 \
  --max_train_steps=400 --mixed_precision="fp16"


Using script: /usr/local/lib/python3.12/dist-packages/diffusers/examples/text_to_image/train_text_to_image_lora.py
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/bin/python3: can't open file '/usr/local/lib/python3.12/dist-packages/diffusers/examples/text_to_image/train_text_to_image_lora.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/accelerate", line 7, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/usr/local/lib/python3.12/dist-packages/accelerate/commands/lau

# Apply the trained LoRA and export 2+ styled images

In [ ]:
# === Apply the trained LoRA and export 2+ styled images (robust, no-crash) ===

# 1) Ensure PEFT is available for LoRA loading
%pip -q install "peft==0.13.2"

import gc, os, glob, torch, importlib
from pathlib import Path
from PIL import Image

# 2) (Re)build the pipeline so the LoRA loader sees PEFT now
try:
    pipe.to("cpu"); del pipe; gc.collect()
except Exception:
    pass

from diffusers import StableDiffusionPipeline
MODEL_ID = "runwayml/stable-diffusion-v1-5"
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)
if device == "cuda":
    pipe.enable_attention_slicing()
    pipe.to(device)

# 3) Try to load LoRA weights if they exist (otherwise we fall back gracefully)
LORA_DIR = Path("lora_weights")
LORA_DIR.mkdir(exist_ok=True, parents=True)

# Refresh the LoRA loader module so it picks up the new PEFT install
import diffusers.loaders.lora_pipeline as _lora_loader
importlib.reload(_lora_loader)

weight_files = sorted(list(LORA_DIR.glob("*.safetensors"))) or sorted(list(LORA_DIR.glob("*.bin")))
adapter_name = "proj_lora"
loaded_lora = False

if len(weight_files) >= 1:
    # Prefer a single, obvious file; if multiple, take the first
    weight_name = weight_files[0].name
    try:
        pipe.load_lora_weights(str(LORA_DIR), adapter_name=adapter_name, weight_name=weight_name)
        if hasattr(pipe, "set_adapters"):
            pipe.set_adapters(adapter_name, adapter_weights=[1.0])  # strength control
        print(f"✅ Loaded LoRA: {weight_name} (adapter='{adapter_name}')")
        loaded_lora = True
    except Exception as e:
        print(f"⚠️ Could not load LoRA ({weight_name}). Using base model instead.\nReason: {e}")
else:
    print("ℹ️ No LoRA weights found in lora_weights/. Using base model outputs.")

# 4) Project-aligned prompts for the styled outputs
styled_prompts = [
    ("multi-agent medical decision diagram in the trained house style, "
     "recruiter → MDT → ICT, teal accents, flat vector, presentation-ready"),
    ("trust game BDI infographic in the trained house style, "
     "belief desire intention panels, reciprocity and risk gauges, clean labels")
]

negative = "low quality, blurry, nsfw, text watermark, logo, distorted text"
OUTPUT_DIR = Path("outputs_sd"); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def save_image(img: Image.Image, name: str) -> str:
    path = str(OUTPUT_DIR / name)
    img.save(path)
    print("saved ->", path)
    return path

# 5) Generate 2+ images (works with or without LoRA)
lora_out_paths = []
suffix = "" if loaded_lora else "_baseline"
for i, p in enumerate(styled_prompts):
    img = pipe(
        prompt=p,
        negative_prompt=negative,
        num_inference_steps=24,
        guidance_scale=7.0,
        height=512,
        width=512
    ).images[0]
    lora_out_paths.append(save_image(img, f"lora_out_{i}{suffix}.png"))

print("✅ Styled outputs:", lora_out_paths)
if not loaded_lora:
    print("ℹ️ These are base-model images because no LoRA weights were found. "
          "Re-run your LoRA training, then re-run this cell to see the LoRA effect.")


Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, due to the lack of support for`float16` operations on this device in PyTorch. Please, remove the `torch_dtype=torch.float16` argument, or use another device for inference.
Pipelines loaded with `dtype=torch.float16` cannot run with `cpu` device. It is not recommended to move them to `cpu` as running them will fail. Please make sure to use an accelerator to run the pipeline in inference, du

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

ℹ️ No LoRA weights found in lora_weights/. Using base model outputs.


  0%|          | 0/24 [00:00<?, ?it/s]

saved -> outputs_sd/lora_out_0_baseline.png


  0%|          | 0/24 [00:00<?, ?it/s]

saved -> outputs_sd/lora_out_1_baseline.png
✅ Styled outputs: ['outputs_sd/lora_out_0_baseline.png', 'outputs_sd/lora_out_1_baseline.png']
ℹ️ These are base-model images because no LoRA weights were found. Re-run your LoRA training, then re-run this cell to see the LoRA effect.


# Task
Explain the error in the selected code. If possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error.

## Clone diffusers repository

### Subtask:
Clone the diffusers GitHub repository to access the training scripts.


**Reasoning**:
The previous attempt to run the training script failed because the script was not found. Cloning the diffusers repository will provide access to the necessary training script.



In [ ]:
!git clone https://github.com/huggingface/diffusers.git

Cloning into 'diffusers'...
remote: Enumerating objects: 106061, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 106061 (delta 270), reused 143 (delta 133), pack-reused 105699 (from 3)
Receiving objects: 100% (106061/106061), 79.57 MiB | 28.58 MiB/s, done.
Resolving deltas: 100% (78552/78552), done.


## Install requirements

### Subtask:
Install the additional requirements needed for training the LoRA model.


**Reasoning**:
Install the training requirements from the requirements.txt file within the examples/text_to_image directory of the cloned repository.



In [ ]:
import os

# Get the current working directory
original_dir = os.getcwd()

# Change the current directory to the cloned diffusers repository examples/text_to_image directory
os.chdir('diffusers/examples/text_to_image')

# Install the training requirements using %pip
%pip -q install -r requirements.txt

# Change the directory back to the original working directory
os.chdir(original_dir)

print("Training requirements installed.")

Training requirements installed.


## Train lora

### Subtask:
Run the LoRA training script from the cloned repository.


**Reasoning**:
The previous attempt to run the training script failed because the script was not found at the specified path. I need to execute the LoRA training script from the correct location within the cloned diffusers repository using `accelerate launch` with the specified arguments.



In [ ]:
import os

# Get the path to the training script within the cloned repository
script_path = "/content/diffusers/examples/text_to_image/train_text_to_image_lora.py"

# Execute the training script using accelerate launch
!accelerate launch {script_path} \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="lora_data" \
  --output_dir="lora_weights" \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --mixed_precision="fp16"


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Traceback (most recent call last):
  File "/content/diffusers/examples/text_to_image/train_text_to_image_lora.py", line 39, in <module>
    from peft import LoraConfig
  File "/usr/local/lib/python3.12/dist-packages/peft/__init__.py", line 17, in <module>
    from .auto import (
  File "/usr/local/lib/python3.12/dist-packages/peft/auto.py", line 31, in <module>
    from .config import PeftConfig
  File "/usr/local/lib/python3.12/dist-packages/peft/config.py", line 24, in <module>
    from .utils import CONFIG_NAME, PeftType, TaskType
  File "/usr/local/lib/python3.12/dist-packages/peft/utils/__init__.py", line

**Reasoning**:
The LoRA training script failed to run due to an `ImportError: cannot import name 'clear_device_cache' from 'accelerate.utils.memory'`. This indicates a version incompatibility between the installed `peft` and `accelerate` libraries. To fix this, I need to install a version of `accelerate` that is compatible with the installed `peft` (0.13.2). Based on the typical dependency relationships, a slightly older version of `accelerate` might resolve this. I will try installing `accelerate==0.28.0`.



In [ ]:
# Install a compatible version of accelerate
%pip install -q accelerate==0.28.0

**Reasoning**:
I have installed a potentially compatible version of `accelerate`. Now I need to re-attempt running the LoRA training script from the cloned repository using the same arguments as before.



In [ ]:
import os

# Get the path to the training script within the cloned repository
script_path = "/content/diffusers/examples/text_to_image/train_text_to_image_lora.py"

# Execute the training script using accelerate launch
!accelerate launch {script_path} \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="lora_data" \
  --output_dir="lora_weights" \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --mixed_precision="fp16"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
Traceback (most recent call last):
  File "/content/diffusers/examples/text_to_image/train_text_to_image_lora.py", line 39, in <module>
    from peft import LoraConfig
  File "/usr/local/lib/python3.12/dist-packages/peft/__init__.py", line 17, in <module>
    from .auto import (
  File "/usr/local/lib/python3.12/dist-packages/peft/auto.py", line 31, in <module>
    from .config import PeftConfig
  File "/usr/local/lib/python3.12/dist-packages/peft/config.py", line 24, in <module>
    from .utils import CONFIG_NAME, PeftType, TaskType
  File "/usr/local/lib/python3.12/dist-packages/peft/utils/__init__.py", line

**Reasoning**:
The training script still fails with the same `ImportError` even after downgrading `accelerate`. This suggests the issue might be with the specific combination of `peft` (0.13.2) and the `diffusers` examples script I'm using. To proceed with the training subtask, I will try installing a different version of `peft` that is more likely to be compatible with the available `accelerate` versions in the environment. I will try installing `peft==0.5.0` as it is often used with earlier `diffusers` examples.



In [ ]:
# Install a different version of peft
%pip install -q peft==0.5.0

**Reasoning**:
I have installed a different version of `peft`. Now I need to re-attempt running the LoRA training script from the cloned repository using the same arguments as before. This is the final attempt to get the training script to run successfully within the constraints of this subtask.



In [ ]:
import os

# Get the path to the training script within the cloned repository
script_path = "/content/diffusers/examples/text_to_image/train_text_to_image_lora.py"

# Execute the training script using accelerate launch
!accelerate launch {script_path} \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="lora_data" \
  --output_dir="lora_weights" \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_warmup_steps=0 \
  --max_train_steps=400 \
  --mixed_precision="fp16"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-10-14 04:08:21.808510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760414901.835707   10615 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760414901.844202   10615 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760414901.877793   10615 computation_plac

## Load lora weights

### Subtask:
Load the trained LoRA weights into the Stable Diffusion pipeline.


**Reasoning**:
Install the specified version of PEFT and reload the lora_pipeline module to ensure the PEFT backend is recognized. Then, list the contents of the lora_weights directory to check for the trained weights and attempt to load them into the pipeline.



In [ ]:
# 1) Install PEFT (compatible with diffusers==0.31.0 / transformers>=4.46)
# Trying peft==0.5.0 again as suggested, though previous attempts failed.
%pip -q install "peft==0.5.0"

# 2) Reload the LoRA loader so it sees that PEFT is available *now*
from importlib import reload
import diffusers.loaders.lora_pipeline as _lora_loader
reload(_lora_loader)

# 3) Sanity-check: list your LoRA weight files
import os, glob
print("LoRA dir:", os.path.abspath("lora_weights"))
weight_files = [os.path.basename(p) for p in glob.glob("lora_weights/*.safetensors")]
print("Files:", os.listdir("lora_weights"))
print("Safetensors files found:", weight_files)


# 4) Load LoRA weights (auto-detect common filenames)
# If your training wrote a single *.safetensors file, this will find it.
kwargs = {}
if len(weight_files) == 1:
    kwargs["weight_name"] = weight_files[0]   # e.g., "adapter_model.safetensors" or similar
elif len(weight_files) > 1:
    print("Warning: Multiple .safetensors files found. Loading all of them.")
elif len(weight_files) == 0:
    print("Error: No .safetensors files found in lora_weights directory.")


# Optionally name the adapter (useful if you’ll load multiple later)
adapter_name = "proj_lora"

# Check if the pipe object exists and has the load_lora_weights method
if 'pipe' in locals() and hasattr(pipe, 'load_lora_weights'):
    try:
        pipe.load_lora_weights(
            "lora_weights",
            adapter_name=adapter_name,
            **kwargs
        )

        # 5) Activate the adapter (scale can be 0.0–1.0; default 1.0)
        if hasattr(pipe, "set_adapters"):
            pipe.set_adapters(adapter_name, adapter_weights=[1.0])

        # 6) Print a confirmation message
        print("✅ LoRA loaded via PEFT backend as:", adapter_name)
    except Exception as e:
        print(f"Error loading LoRA weights: {e}")
else:
    print("Error: 'pipe' object not found or does not have 'load_lora_weights' method.")



LoRA dir: /content/lora_weights
Files: []
Safetensors files found: []
Error: No .safetensors files found in lora_weights directory.
Error loading LoRA weights: PEFT backend is required for this method.


## Generate images with lora

### Subtask:
Generate images using the pipeline with the loaded LoRA weights.


## Train lora retry 1

### Subtask:
Run the LoRA training script from the cloned repository.


**Reasoning**:
The previous attempt to run the training script failed because the script file was not found at the expected path. The file listing shows that the `diffusers` repository was cloned to `/content/diffusers`. I need to construct the correct path to the training script within the cloned repository and attempt to run it again. I will also add a check to confirm the script exists before attempting to execute it.



In [ ]:
import os

# Get the path to the training script within the cloned repository
script_path = "/content/diffusers/examples/text_to_image/train_text_to_image_lora.py"

# Check if the script exists
if not os.path.exists(script_path):
    print(f"Error: Training script not found at {script_path}")
else:
    print(f"Found script at: {script_path}")
    # Execute the training script using accelerate launch
    !accelerate launch {script_path} \
      --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
      --instance_data_dir="lora_data" \
      --output_dir="lora_weights" \
      --train_batch_size=1 \
      --gradient_accumulation_steps=4 \
      --learning_rate=1e-4 \
      --lr_warmup_steps=0 \
      --max_train_steps=400 \
      --mixed_precision="fp16"


Found script at: /content/diffusers/examples/text_to_image/train_text_to_image_lora.py
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-10-14 04:09:33.493289: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760414973.513068   10970 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760414973.519154   10970 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when

## Load lora weights retry 1

### Subtask:
Load the trained LoRA weights into the Stable Diffusion pipeline.


**Reasoning**:
Install the specified version of peft and reload the lora_pipeline module. Then, list the files in the lora_weights directory, check for the existence of the pipe object and the load_lora_weights method, and if found, attempt to load the lora weights with the specified adapter name and potential weight name if a single safetensors file is found. Finally, activate the adapter if the set_adapters method exists and print a confirmation or error message.



In [ ]:
# 1) Install PEFT (compatible with diffusers==0.31.0 / transformers>=4.46)
# Trying peft==0.4.0 based on instructions.
%pip -q install "peft==0.4.0"

# 2) Reload the LoRA loader so it sees that PEFT is available *now*
from importlib import reload
import diffusers.loaders.lora_pipeline as _lora_loader
reload(_lora_loader)

# 3) Sanity-check: list your LoRA weight files
import os, glob
print("LoRA dir:", os.path.abspath("lora_weights"))
weight_files = [os.path.basename(p) for p in glob.glob("lora_weights/*.safetensors")]
print("Files in lora_weights:", os.listdir("lora_weights"))
print(".safetensors files found:", weight_files)


# 4) Load LoRA weights (auto-detect common filenames)
# If your training wrote a single *.safetensors file, this will find it.
kwargs = {}
if len(weight_files) == 1:
    kwargs["weight_name"] = weight_files[0]   # e.g., "adapter_model.safetensors" or similar
elif len(weight_files) > 1:
    print("Warning: Multiple .safetensors files found. Attempting to load all.")
    # Depending on how load_lora_weights handles multiple files, this might need adjustment
    # For now, we'll proceed with just the directory
    kwargs = {} # Reset kwargs to avoid passing weight_name
elif len(weight_files) == 0:
    print("Error: No .safetensors files found in lora_weights directory.")


# Optionally name the adapter (useful if you’ll load multiple later)
adapter_name = "proj_lora"

# Check if the pipe object exists and has the load_lora_weights method
if 'pipe' in locals() and hasattr(pipe, 'load_lora_weights'):
    try:
        # Check if weight_files is not empty before attempting to load
        if len(weight_files) > 0:
            pipe.load_lora_weights(
                "lora_weights",
                adapter_name=adapter_name,
                **kwargs
            )

            # 5) Activate the adapter (scale can be 0.0–1.0; default 1.0)
            if hasattr(pipe, "set_adapters"):
                pipe.set_adapters(adapter_name, adapter_weights=[1.0])

            # 6) Print a confirmation message
            print("✅ LoRA loaded via PEFT backend as:", adapter_name)
        else:
            print("LoRA weights not loaded because no .safetensors files were found.")

    except Exception as e:
        print(f"Error loading LoRA weights: {e}")
else:
    print("Error: 'pipe' object not found or does not have 'load_lora_weights' method.")


LoRA dir: /content/lora_weights
Files in lora_weights: []
.safetensors files found: []
Error: No .safetensors files found in lora_weights directory.
LoRA weights not loaded because no .safetensors files were found.


## Summary:

### Data Analysis Key Findings

*   The primary error encountered during the process was an `ImportError` when attempting to run the LoRA training script, indicating a dependency conflict.
*   The training script specifically required `diffusers` version 0.36.0.dev0 or newer, installed from source, while the available environment only had version 0.31.0.
*   Attempts to resolve the `ImportError` by installing different versions of `accelerate` and `peft` were unsuccessful.
*   Due to the failure of the training script, no LoRA weight files (`.safetensors`) were generated in the `lora_weights` directory.
*   Consequently, subsequent steps to load the LoRA weights and generate images with the fine-tuned model failed because the necessary weight files were missing.

### Insights or Next Steps

*   To successfully run the LoRA training script, the environment needs to be configured to meet the specific `diffusers` version requirement, which involves a source installation of a newer version.
*   Alternatively, the training script itself might need modification to be compatible with the available version of `diffusers` or other libraries, or an older version of the script that is compatible with the installed libraries could be used.


In [ ]:
!pip install --upgrade nbconvert

!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True \ --to notebook --inplace Week_7_Task1_Stable_diffusion.ipynb

usage: jupyter-nbconvert [-h] [--debug] [--show-config] [--show-config-json]
                         [--generate-config] [-y] [--execute] [--allow-errors]
                         [--stdin] [--stdout] [--inplace] [--clear-output]
                         [--coalesce-streams] [--no-prompt] [--no-input]
                         [--allow-chromium-download]
                         [--disable-chromium-sandbox] [--show-input]
                         [--embed-images] [--sanitize-html]
                         [--log-level NbConvertApp.log_level]
                         [--config NbConvertApp.config_file]
                         [--to NbConvertApp.export_format]
                         [--template TemplateExporter.template_name]
                         [--template-file TemplateExporter.template_file]
                         [--theme HTMLExporter.theme]
                         [--sanitize_html HTMLExporter.sanitize_html]
                         [--writer NbConvertApp.writer_class]
   

In [ ]:
from google.colab import files
files.download("Week_7_Task1_Stable_diffusion.ipynb")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>